# Extraction of extracellular matrix from histological microscopic images of the liver using machine learning methods
Jan Burian

## Installing dependencies and import needed modules

### ImageSplitterMerger

In [1]:
# Installations of needed modules
!pip install openslide-python openslide-bin imagecodecs loguru read_roi czifile
!pip install git+https://github.com/mjirik/imma.git

# WSI tools installation
!pip install git+https://github.com/mjirik/wsitools.git

# Modules import
import matplotlib.pyplot as plt
import numpy as np
import imma
import requests
import sys
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openslide-python: filename=openslide_python-1.3.1-cp310-cp310-linux_x86_64.whl size=33550 sha256=ddafd24167c940366dc4d89f2b5bcb320e206246b1ccafca020f24ad383eca1f
  Stored in directory: /root/.cache/pip/wheels/79/79/fa/29a0087493c69dff7fd0b70fab5d6771002a531010161d2d97
  Created wheel for read_roi: filename=read_roi-1.6.0-py2.py3-none-any.whl size=8017 sha256=55116d119a372bc49b5ce54de278f73db1f53c94adeb1300fcbc245a5698def8
  Stored in directory: /root/.cache/pip/wheels/09/67/72/4903ef58791d5330f9946373f84126b8b79027879ac828702e
Successfully built openslide-python read_roi


### Main repository

In [2]:
!git clone https://github.com/janburian/Masters_thesis.git

Cloning into 'Masters_thesis'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 288 (delta 21), reused 22 (delta 4), pack-reused 230
Receiving objects: 100% (288/288), 380.84 MiB | 20.69 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Updating files: 100% (42/42), done.


Methods import

In [3]:
sys.path.insert(0, os.path.abspath('./Masters_thesis/src'))

In [11]:
from ECM_removal_methods import (
    download_czi_file,
    download_public_gdrive_folder,
    jaccard_coef,
    jaccard_coef_loss,
    do_inference_unet,
    do_inference_mask_rcnn,
    apply_dilation,
    do_inpainting_biharmonic,
    remove_ECM_unet,
    # process_tile_unet,
    # process_tile_mask_rcnn
)

### Detectron2

In [6]:
!python -m pip install pyyaml==5.1
import distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 3.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15676, done.
remote: Counting objects: 100% (399/399), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 15676 (delta 176), reused 287 (delta 95), pack-reused 15277
Receiving objects: 100% (15676/15676), 6.51 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (11288/11288), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 971.3 kB/s eta 0:

In [7]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from pathlib import Path

import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# U-Net
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from skimage.transform import resize

# Inpainting
from skimage.restoration import inpaint
from skimage.morphology import dilation

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


## CZI file

In [8]:
# Possible filenames: J7_5_a.czi, J7_25_a_ann0004.czi, J8_8_a.czi
filename = "J7_5_a.czi"
url_path = "https://github.com/janburian/Masters_thesis/raw/main/data_czi/"

download_czi_file(url_path, filename)

## Segmentation setup

### Mask R-CNN (Detectron2)

In [9]:
mask_rcnn_weights_url = "https://drive.google.com/drive/folders/1j7Mn7EREHOGCKJDk77NRjeRxboUHrEti?usp=sharing"
local_download_path = "/content/mask_rcnn_weights"

download_public_gdrive_folder(mask_rcnn_weights_url, local_download_path)

1j7Mn7EREHOGCKJDk77NRjeRxboUHrEti?usp=sharing


Retrieving folder contents


Retrieving folder 1-UDHDGhWHAoJjsfgk3T6w2PAvVwCADTL output
Processing file 1-erwv9_vmRbOknEHBy1s5NUFLvFDaWa0 coco_instances_results.json
Processing file 1-VNSgpW_L3QyKFht-ViEY1sq0CADmR1V events.out.tfevents.1715590033.913fe913d14b.414.0
Processing file 1-qkaOHRxvy5uFaylzUgsMdP02dVKmjpi instances_predictions.pth
Processing file 1-v7V3sdXHZsQscbHpGCo7-B6akDpf_lo last_checkpoint
Processing file 1-VsgJTrCdDmO9-12mg6AHcKPlNSQXBqP metrics.json
Processing file 1-av2pVjxdowsWsGRTDnwVsiWMU09Y6Cs model_final.pth
Retrieving folder 1--IJjLvXw16ysL26oYZ73YmpAn6OHCKM R50-FPN
Processing file 1IBTkBOgQdOntBLS7801nTsjGz8KtJz38 config.yaml
Processing file 1-D049jvWI0p3UoHSO5kRFGIIClaq2vrq events.out.tfevents.1715672909.60e05f48b7f5.2466.0
Processing file 1-MQem5DoPkTirYtLD3LEHPIw3exUe-jd last_checkpoint
Processing file 1-E7o6bHGhxyI4j3cBRNfYzOl2IZ8A1-s metrics.json
Processing file 1-3JQNwqTO5n4_ZyMP4mfKwblwp0sgWpo ResNet50-FPN.pth
Retrieving folder 1AvZFnYnoGSqybgMxGbUjHJs9R631tocB R101-FPN
Processing f

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-erwv9_vmRbOknEHBy1s5NUFLvFDaWa0
To: /content/mask_rcnn_weights/output/coco_instances_results.json
100%|██████████| 61.8k/61.8k [00:00<00:00, 20.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-VNSgpW_L3QyKFht-ViEY1sq0CADmR1V
To: /content/mask_rcnn_weights/output/events.out.tfevents.1715590033.913fe913d14b.414.0
100%|██████████| 28.5k/28.5k [00:00<00:00, 11.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-qkaOHRxvy5uFaylzUgsMdP02dVKmjpi
To: /content/mask_rcnn_weights/output/instances_predictions.pth
100%|██████████| 38.8k/38.8k [00:00<00:00, 62.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-v7V3sdXHZsQscbHpGCo7-B6akDpf_lo
To: /content/mask_rcnn_weights/output/last_checkpoint
100%|██████████| 15.0/15.0 [00:00<00:00, 47.9kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-VsgJTrCdDmO9-12

In [17]:
# Setup config and corresponding own weights
MASK_RCNN_MODEL = "/content/mask_rcnn_weights/R101-FPN/config.yaml"

cfg = get_cfg()
cfg.merge_from_file(MASK_RCNN_MODEL) # Config file
cfg.MODEL.WEIGHTS = "/content/mask_rcnn_weights/R101-FPN/ResNet101-FPN.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set a custom testing threshold
# cfg.MODEL.DEVICE = "cpu" # use if you do not have available GPU
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only one class
predictor = DefaultPredictor(cfg)

[05/14 12:18:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/mask_rcnn_weights/R101-FPN/ResNet101-FPN.pth ...


### UNet

In [14]:
unet_model_path = '/content/Masters_thesis/models/cell nuclei/20240415-13_06_unet_cell_nuclei_150epochs.keras'
model_unet = load_model(unet_model_path)

In [15]:
unet_model_ECM_path = '/content/Masters_thesis/models/ECM/unet_ECM_final_50epochs.keras'
model_unet_ECM = load_model(unet_model_ECM_path)

## Implementing *process_tile()* methods

In [12]:
def process_tile_unet(tile: np.array) -> np.array:
  orig_tile_shape = tile.shape
  mask = do_inference_unet(tile, model_unet, orig_tile_shape)
  # mask = create_mask_from_inferences(res_inference)
  # print(type(mask))
  if np.any(mask):  # determine whether in mask are white pixels
    mask_dil = apply_dilation(mask, 5)
    # plt.imshow(mask_dil)
    # plt.show()
    img_removed_cell_nuclei = do_inpainting_biharmonic(tile, mask_dil)
    img_removed_extracellular_matrix = remove_ECM_unet(img_removed_cell_nuclei, model_unet_ECM, orig_tile_shape)
    # plt.imshow(img_removed_extracellular_matrix)
    # plt.show()
    # print(img_removed_extracellular_matrix)
    # return img_removed_cell_nuclei
    # img_removed_extracellular_matrix_new_color = create_pink_contours(img_removed_extracellular_matrix)
    # img_removed_extracellular_matrix = convert_grayscale_to_RGB(img_removed_extracellular_matrix)
    # plt.imshow(img_removed_extracellular_matrix_new_color)
    # plt.show()
    # print(img_removed_extracellular_matrix_new_color)

    # fig, axes = plt.subplots(1, 4, figsize=(12, 6))
    # axes[0].imshow(tile)
    # axes[0].set_title("Original Tile")
    # axes[0].axis('off')
    # axes[1].imshow(mask_dil, cmap="gray")
    # axes[1].set_title("Mask")
    # axes[1].axis('off')
    # axes[2].imshow(img_removed_cell_nuclei)
    # axes[2].set_title("Tile with removed cell nuclei")
    # axes[2].axis('off')
    # axes[3].imshow(img_removed_extracellular_matrix)
    # axes[3].set_title("Removed ECM")
    # axes[3].axis('off')
    # plt.show()

    # plt.imshow(img_removed_extracellular_matrix)
    # plt.show()
    # print(img_removed_extracellular_matrix.shape)

    return img_removed_extracellular_matrix

    # return np.stack((img_removed_extracellular_matrix,) * 3, axis=-1) # TODO

  else: # no mask in tile; returning the original tile
      # plt.imshow(tile)
      # plt.show()
      # print(tile)
      # return tile
      tile_normalized = tile / 255.0
      img_removed_extracellular_matrix = remove_ECM_unet(tile_normalized, model_unet_ECM, orig_tile_shape)
      # img_removed_extracellular_matrix_new_color = create_pink_contours(img_removed_extracellular_matrix)
      # img_removed_extracellular_matrix_new_color = convert_grayscale_to_RGB(img_removed_extracellular_matrix)

      # plt.imshow(img_removed_extracellular_matrix)
      # plt.show()
      # print('TEST: ' + str(img_removed_extracellular_matrix.dtype))
      return img_removed_extracellular_matrix

      # return np.stack((img_removed_extracellular_matrix,) * 3, axis=-1) # TODO



def process_tile_mask_rcnn(tile: np.array) -> np.array:
  orig_tile_shape = tile.shape
  mask = do_inference_mask_rcnn(tile, predictor)
  if np.any(mask):  # determine whether in mask are white pixels
    mask_dil = apply_dilation(mask, 5)
    # plt.imshow(mask_dil)
    # plt.show()
    img_removed_cell_nuclei = do_inpainting_biharmonic(tile, mask_dil)
    img_removed_extracellular_matrix = remove_ECM_unet(img_removed_cell_nuclei, model_unet_ECM, orig_tile_shape)
    # print(img_removed_cell_nuclei.dtype)

    # print(tile_removed_cell_nuclei)
    # fig, axes = plt.subplots(1, 4, figsize=(12, 6))
    # axes[0].imshow(tile)
    # axes[0].set_title("Original Tile")
    # axes[0].axis('off')
    # axes[1].imshow(mask_dil, cmap="gray")
    # axes[1].set_title("Mask")
    # axes[1].axis('off')
    # axes[2].imshow(img_removed_cell_nuclei)
    # axes[2].set_title("Tile with removed cell nuclei")
    # axes[2].axis('off')
    # axes[3].imshow(img_removed_extracellular_matrix)
    # axes[3].set_title("Removed ECM")
    # axes[3].axis('off')
    # plt.show()

    # print(img_removed_cell_nuclei)
    return img_removed_extracellular_matrix

  else: # no mask in tile; returning the original tile
      # plt.imshow(tile)
      # plt.show()
      tile_normalized = tile / 255.0
      # plt.imshow(remove_ECM_unet(tile_normalized, model_unet_ECM, orig_tile_shape))
      # plt.show()
      return remove_ECM_unet(tile_normalized, model_unet_ECM, orig_tile_shape)

## Final part

In [ ]:
from wsitools.tile_image import ImageSplitterMerger

# Create an ImageSplitterMerger instance with the specified parameters
# U-Net cell nuclei segmentation
# image = ImageSplitterMerger(filename, tilesize_px=150, overlap_px=15, pixelsize_mm=[0.001, 0.001], # tilesize_px=150, pixelsize_mm=[0.001, 0.001] with overlap_px=15 is probably best
#                             fcn=process_tile_unet)

# Mask R-CNN cell nuclei segmentation
image = ImageSplitterMerger(filename, tilesize_px=150, overlap_px=15, pixelsize_mm=[0.001, 0.001], # tilesize_px=150, pixelsize_mm=[0.001, 0.001] with overlap_px=15 is probably best
                            fcn=process_tile_mask_rcnn)

# Split and merge the image, applying the specified tile processing function
merged_image_processed = image.split_and_merge_image()

# Display the input and merged images using Matplotlib
plt.imshow(merged_image_processed)
plt.title("Merged Processed Image")
plt.show()

2024-05-14 12:18:39.360 | DEBUG    | wsitools.image:__init__:412 - fs_enc: utf-8
2024-05-14 12:18:39.361 | DEBUG    | wsitools.image:__init__:413 - Reading file J7_5_a.czi
2024-05-14 12:18:39.386 | DEBUG    | wsitools.image:_set_properties_czi:342 - nzi pixelsize  8.772367043811993e-07x8.772367043811993e-07 [m]
2024-05-14 12:18:39.390 | DEBUG    | wsitools.image:read_annotations:723 - Reading the annotation J7_5_a.czi


True


Splitting and Processing Tiles:   0%|          | 0/8600 [00:00<?, ?it/s]

1/1 [==============================] - 0s 26ms/step


Splitting and Processing Tiles:   0%|          | 1/8600 [00:01<4:13:37,  1.77s/it]

...

1/1 [==============================] - 0s 30ms/step


Splitting and Processing Tiles:   9%|▉         | 802/8600 [06:24<59:51,  2.17it/s]

1/1 [==============================] - 0s 24ms/step


Splitting and Processing Tiles:   9%|▉         | 803/8600 [06:24<1:02:04,  2.09it/s]

1/1 [==============================] - ETA: 0s

In [ ]:
plt.imsave("output.png", merged_image_processed)

In [ ]:
from wsitools.tile_image import ImageSplitterMerger

orig_image = ImageSplitterMerger(filename, tilesize_px=3000, overlap_px=0, pixelsize_mm=[0.001, 0.001],
                            fcn=None)

# Split and merge the image (in this case we use no tile processing function) - we
# only want to visualize the original image
orig_merged_image = orig_image.split_and_merge_image()

# plt.imshow(orig_merged_image)
# plt.title("Original Image")
# plt.show()

### Image comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(orig_merged_image)
axes[0].set_title("Original Image")
axes[0].axis('off')
axes[1].imshow(merged_image_processed)
axes[1].set_title("Processed Merged Image")
axes[1].axis('off')
plt.show()

## Hardware

In [ ]:
!nvidia-smi
!cat /proc/cpuinfo
!cat /proc/meminfo